# 01 - 数据加载与初步观察

## 🎯 学习目标
- 掌握Pandas读取各类数据文件
- 理解数据的基本结构（行、列、类型）
- 学会快速诊断数据质量问题

## 📚 知识点
1. `pd.read_csv()` 参数详解
2. DataFrame基本属性：shape, dtypes, info(), describe()
3. 内存优化：降低数据类型内存占用
4. 缺失值初步检查

---
## 1. 环境准备

In [ ]:
# 导入基础库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 配置
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']  # 中文显示
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

# 显示设置
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

print("✅ 环境准备完成")

---
## 2. 数据加载

### 💡 学习要点
- 不同文件格式的读取方法
- 重要参数：`sep`, `encoding`, `parse_dates`, `na_values`
- 读取大文件的技巧：`chunksize`, `usecols`

In [ ]:
# 数据路径
DATA_PATH = '../../data/raw/'

# 读取训练集
train = pd.read_csv(DATA_PATH + 'train.csv')

# 读取测试集
test = pd.read_csv(DATA_PATH + 'test.csv')

print(f"训练集形状: {train.shape}")
print(f"测试集形状: {test.shape}")

### 🔍 快速查看前几行

In [ ]:
train.head()

In [ ]:
test.head()

---
## 3. 数据结构理解

### 💡 学习要点
- `shape`: (行数, 列数)
- `dtypes`: 每列的数据类型
- `info()`: 综合信息，包括缺失值
- `columns`: 列名列表

In [ ]:
print("="*50)
print("训练集基本信息")
print("="*50)
train.info()

In [ ]:
# 数据类型统计
print("\n数据类型分布:")
print(train.dtypes.value_counts())

In [ ]:
# 分离数值型和类别型特征
numeric_features = train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = train.select_dtypes(include=['object']).columns.tolist()

print(f"\n数值型特征 ({len(numeric_features)}个): {numeric_features}")
print(f"\n类别型特征 ({len(categorical_features)}个): {categorical_features}")

---
## 4. 描述性统计

### 💡 学习要点
- `describe()`: 数值型特征的统计摘要
- 关注：均值、标准差、分位数、极值
- 识别潜在异常值（极值过大/过小）

In [ ]:
# 数值型特征统计
train.describe()

In [ ]:
# 类别型特征统计
train.describe(include=['object'])

---
## 5. 缺失值分析

### 💡 学习要点
- `isnull().sum()`: 每列缺失值数量
- 缺失率计算
- 可视化缺失模式

In [ ]:
def missing_values_table(df):
    """生成缺失值统计表"""
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * mis_val / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: '缺失值数量', 1: '缺失率(%)'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '缺失率(%)', ascending=False).round(2)
    print(f"数据集共有 {df.shape[1]} 列")
    print(f"其中 {mis_val_table_ren_columns.shape[0]} 列有缺失值")
    return mis_val_table_ren_columns

missing_train = missing_values_table(train)
missing_train

In [ ]:
# 可视化缺失值
if len(missing_train) > 0:
    plt.figure(figsize=(10, 6))
    plt.barh(missing_train.index, missing_train['缺失率(%)'])
    plt.xlabel('缺失率 (%)')
    plt.title('特征缺失率')
    plt.tight_layout()
    plt.show()

---
## 6. 目标变量分析

### 💡 学习要点
- 分类问题：类别分布是否均衡
- 回归问题：目标变量的分布形态

In [ ]:
# 假设目标变量列名为 'target'
TARGET_COL = 'target'  # 根据实际情况修改

if TARGET_COL in train.columns:
    print(f"目标变量 '{TARGET_COL}' 统计:")
    print(train[TARGET_COL].describe())
    print(f"\n类别分布:")
    print(train[TARGET_COL].value_counts())
    print(f"\n类别占比:")
    print(train[TARGET_COL].value_counts(normalize=True))

---
## 7. 内存优化

### 💡 学习要点
- 通过降低数值类型精度减少内存占用
- int64 → int8/int16/int32
- float64 → float32

In [ ]:
def reduce_mem_usage(df, verbose=True):
    """内存优化函数"""
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'内存使用从 {start_mem:.2f} MB 降至 {end_mem:.2f} MB '
              f'(减少 {100 * (start_mem - end_mem) / start_mem:.1f}%)')
    return df

# 应用内存优化
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

---
## 📝 本节总结

### 已完成的工作
- [ ] 成功加载训练集和测试集
- [ ] 了解数据的基本结构（行数、列数、类型）
- [ ] 识别数值型和类别型特征
- [ ] 分析缺失值情况
- [ ] 初步了解目标变量分布
- [ ] 优化内存使用

### 发现的问题
记录你发现的数据质量问题：
1. 
2. 
3. 

### 下一步计划
- 单变量分析：深入了解每个特征的分布
- 双变量分析：探索特征与目标的关系

---
## 💾 保存处理后的数据

In [ ]:
# 保存到processed目录
train.to_pickle('../../data/processed/train_initial.pkl')
test.to_pickle('../../data/processed/test_initial.pkl')
print("✅ 数据已保存")